In [ ]:
# !pip install keras_cv_attention_models

In [ ]:
import os
from tensorflow import keras
import losses, train, models
import tensorflow as tf
import keras_cv_attention_models
from keras_cv_attention_models import efficientnet
import tensorflow_addons as tfa

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Remove the below for better accuracies and keep it for faster training
keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
basic_model = efficientnet.EfficientNetV2S(input_shape=(112, 112, 3), num_classes=0, drop_connect_rate=0.2, pretrained="imagenet", first_strides=1)
basic_model = models.buildin_models(basic_model, dropout=0.2, emb_shape=512, output_layer='F', bn_momentum=0.9, bn_epsilon=1e-5, add_pointwise_conv=True, pointwise_conv_act="swish", scale=True, use_bias=False)

In [ ]:
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

In [ ]:
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='EfficientFaceV2S.h5',
    basic_model=basic_model, model=None, lr_base=0.01, lr_decay=0.5, lr_decay_steps=50, lr_min=1e-6, lr_warmup_steps=3,
    batch_size=512, random_status=100, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = tfa.optimizers.AdamW(learning_rate=1e-2, weight_decay=5e-4)

In [ ]:
sch = [
    {"loss": losses.AdaFaceLoss(scale=16), "epoch": 6, "optimizer": optimizer},
    {"loss": losses.AdaFaceLoss(scale=32), "epoch": 8},
    {"loss": losses.AdaFaceLoss(scale=64), "epoch": 46},
]
tt.train(sch, 0)